In [5]:
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd

# from plotly.offline import init_notebook_mode
# init_notebook_mode(connected = True)

In [6]:
df = pd.read_parquet('../data/SO_2014_2022.pq')
start_salary = df.query('YearsCodePro < 2')['Salary'].mean()
avg_salary = ((df.groupby(['YearsCodePro', 'Gender'])['Salary'].mean() / start_salary - 1) * 100).reset_index()

In [7]:
smoothed_avg = pd.DataFrame(index=list(range(1, 50)), columns=['male', 'female'])

for gender in ['male', 'female']:
    x = list(avg_salary.query(f"Gender == '{gender}'").drop(columns=['Gender']).rolling(3, on='YearsCodePro'))
    smoothed = pd.Series(row.mean()['Salary'] for row in x[2:])
    smoothed.index += 1
    smoothed_avg[gender] = smoothed

smoothed_avg.reset_index(names='YearsCodePro', inplace=True)

In [8]:
fig = px.line(smoothed_avg, x="YearsCodePro", y=['male', 'female'],
              range_x=[1, 40], range_y=[0, 400])
fig.update_layout(xaxis_title="Years coded professionally", yaxis_title="Average increase in salary")

fig.show()